In [3]:
import keras
from keras.models import load_model

In [ ]:
# import os 
# import xml.etree.ElementTree as ET

# def convert_box(size, box):
#     dw, dh = 1. / size[0], 1. / size[1]
#     x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
#     return x * dw, y * dh, w * dw, h * dh

# def convert_voc_to_yolo():
#     for anno in os.listdir('./data/labels'):
#         if anno.split('.')[1] == 'xml':
#             file_name = anno.split('.')[0]
#             out_file = open(f'./data/labels/{file_name}.txt', 'w')

#             tree = ET.parse(os.path.join('data','labels', anno))
#             root = tree.getroot()
#             size = root.find('size')        
#             w = int(size.find('width').text)
#             h = int(size.find('height').text)

#             names = ['turn_right', 'turn_left', 'dont_turn_right', 'dont_turn_left', 'straight', 'stop']

#             for obj in root.iter('object'):
#                 cls = obj.find('name').text
#                 if cls in names and int(obj.find('difficult').text) != 1:
#                     xmlbox = obj.find('bndbox')
#                     bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
#                     cls_id = names.index(cls)  # class id
#                     out_file.write(" ".join([str(a) for a in (cls_id, *bb)]) + '\n')

In [ ]:
# convert_voc_to_yolo()

In [ ]:
%%cmd
cd yolov5
python train.py --img 640 --batch 16 --epochs 100 --data customVOC.yaml --weights yolov5n.pt --workers 2

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
import itertools

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/OBJ/test8.pt', force_reload=True)
model.conf = 0.5
cnn_model = load_model('./CNN_models/CNN-model-020.h5')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\ADMIN/.cache\torch\hub\master.zip
YOLOv5  2024-1-12 Python-3.10.7 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)



Exception: cannot instantiate 'PosixPath' on your system. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.

In [11]:
imgs = os.listdir('./TrafficSignData/completedata/')
class_dict={
    0: 'greenlight',
    1: 'left',
    2: 'noleft',
    3: 'noright',
    4: 'nostraight',
    5: 'redlight',
    6: 'right',
    7: 'stop',
    8: 'straight',
    9: 'yellowlight',
}
count = 0
for i in imgs:
    img = cv2.imread(f'./TrafficSignData/completedata/{i}')
    results = model(img)
    b = results.xyxy[0]
    b = b.cpu().numpy().astype(int)
    for tmp in b:
        top, left, bottom, right, _, _ = tmp
        cropped_box = img[left:right, top:bottom]
        cropped_box = cv2.resize(cropped_box, (50,50))
        cropped_box = np.expand_dims(cropped_box, axis=0)
        label = class_dict[np.argmax(cnn_model.predict(cropped_box))]
        top, left, bottom, right = int(top), int(left), int(bottom), int(right)
        cv2.rectangle(img, (top, left), (bottom, right), (0, 255, 0), 2)  # (0, 255, 0)
        text_position = (top, left - 10)  # Điều chỉnh vị trí của chuỗi văn bản
        cv2.putText(img, label, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    # res = np.squeeze(results.render())
    try:
        os.chdir('./OBJresults')
    except:
        os.mkdir('./OBJresults')
        os.chdir('./OBJresults')
    cv2.imwrite(f'{count}.jpg', img)
    count +=1
    os.chdir('..')

1/1 [==============================] - 0s 24ms/step


In [28]:
imgs = os.listdir('./TrafficSignData/completedata/')

count = 0
for i in imgs:
    img = cv2.imread(f'./TrafficSignData/completedata/{i}')
    results = model(img)
    b = results.xyxy[0]
    b = b.cpu().numpy().astype(int)
    for tmp in b:
        top, left, bottom, right, _, _ = tmp
        cropped_box = img[left:right, top:bottom]
        try:
            os.chdir('./OBJcropped')
        except:
            os.mkdir('./OBJcropped')
            os.chdir('./OBJcropped')
        cv2.imwrite(f'{count}.jpg', cropped_box)
        os.chdir('..')
    res = np.squeeze(results.render())
    try:
        os.chdir('./OBJresults')
    except:
        os.mkdir('./OBJresults')
        os.chdir('./OBJresults')
    cv2.imwrite(f'{count}.jpg', res)
    count +=1
    os.chdir('..')

In [8]:
!python ./yolov5/export.py --weights models/OBJ/test8.pt --include onnx --device 0 --half --imgsz 224

export: data=D:\UIT_Car_Racing\yolov5\data\coco128.yaml, weights=['models/OBJ/test8.pt'], imgsz=[224], batch_size=1, device=0, half=True, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5  v7.0-240-g84ec8b5 Python-3.10.7 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 1772695 parameters, 0 gradients, 4.2 GFLOPs

PyTorch: starting from models\OBJ\test8.pt with output shape (1, 3087, 15) (3.6 MB)

ONNX: starting export with onnx 1.14.1...
ONNX: export success  0.7s, saved as models\OBJ\test8.onnx (3.4 MB)

Export complete (4.2s)
Results saved to D:\UIT_Car_Racing\models\OBJ
Detect:          python detect.py --weights models\OBJ\test8.onnx --half
Validate:        python val.py --weights models\OBJ\test8.onnx --half
PyTorch Hub:

In [11]:
from ultralytics import YOLO

model = YOLO('./models/OBJ/09-11-0240pm(v8).pt')

In [12]:
from ultralytics.utils.plotting import Annotator

imgs = os.listdir('./TrafficSignData/new/')

count = 0
for i in imgs:
    img = cv2.imread(f'./TrafficSignData/new/{i}')
    results = model.predict(img,conf=0.25)
    for r in results:
        annotator = Annotator(img)
        boxes = r.boxes
        for box in boxes:
            
            b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
            c = box.cls
            annotator.box_label(b, model.names[int(c)])
    res = annotator.result()
    try:
        os.chdir('./V8results')
    except:
        os.mkdir('./V8results')
        os.chdir('./V8results')
    cv2.imwrite(f'{count}.jpg', res)
    count +=1
    os.chdir('..')


0: 640x640 1 noright, 1 stop, 1 straight, 57.1ms
Speed: 8.0ms preprocess, 57.1ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 nostraight, 1 stop, 1 straight, 55.0ms
Speed: 7.1ms preprocess, 55.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 left, 1 noright, 1 nostraight, 1 right, 52.8ms
Speed: 5.0ms preprocess, 52.8ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 noright, 1 nostraight, 1 right, 51.8ms
Speed: 6.0ms preprocess, 51.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 noright, 1 nostraight, 1 right, 41.0ms
Speed: 6.0ms preprocess, 41.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 noright, 1 nostraight, 1 right, 25.8ms
Speed: 4.0ms preprocess, 25.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 noright, 1 right, 26.0ms
Speed: 4.0ms preprocess, 26.0ms inference, 2.0ms postpro

In [3]:
import torch

# Kiểm tra xem PyTorch hỗ trợ CUDA không
if torch.cuda.is_available():
    # Lấy số lượng GPU có sẵn
    num_cuda_devices = torch.cuda.device_count()
    print(f"Có {num_cuda_devices} thiết bị CUDA sẵn sàng sử dụng.")

    # Liệt kê thông tin về các thiết bị CUDA
    for i in range(num_cuda_devices):
        device = torch.cuda.get_device_name(i)
        print(f"Thiết bị CUDA {i}: {device}")
else:
    print("Không có thiết bị CUDA sẵn sàng sử dụng.")

Có 1 thiết bị CUDA sẵn sàng sử dụng.
Thiết bị CUDA 0: NVIDIA GeForce RTX 3050 Laptop GPU


In [15]:
from ultralytics import YOLO

model = YOLO('./models/OBJ/09-11-0240pm(v8).pt')

model.export(format='trt')

WARNING  TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.201  Python-3.10.7 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'models\OBJ\09-11-0240pm(v8).pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (6.0 MB)
TensorRT: export failure  0.0s: No module named 'tensorrt'


ModuleNotFoundError: No module named 'tensorrt'